In [1]:
# pytorch optimizer 让动量参与计算，以及手动修改lr

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import numpy as np

import ray
import resnet.models as models
import random,time
from time import sleep
import copy 
import datetime
import argparse


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
import os
import shutil
from torch.utils.tensorboard import SummaryWriter
from filelock import FileLock

/userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

In [5]:
model1 = models.__dict__["resnet"](dataset="cifar10",depth=164)
optimizer1 = optim.SGD(model1.parameters(),
                          lr=0.1,
                          momentum=0.9,
                          weight_decay=1e-4)
model2 = models.__dict__["resnet"](dataset="cifar10",depth=164)
optimizer2 = optim.SGD(model2.parameters(),
                          lr=0.1,
                          momentum=0.9,
                          weight_decay=1e-4)